# Projekt nr 2
#### Maciej Konieczny
#### Kamila Martyniuk-Sienkiewicz
Na rzecz uruchomienia projektu, konieczne jest załadowanie środowiska i biblioteki agentów. Komendy w następnej komórce importują co potrzeba, ALE załadowanie środowiska Hoppera wymaga fizyki mujoco i biblioteki imageio:
- pip install mujoco
- pip install imageio

Przy instalacji mujoco w wersji 3 lub nowszej wystepuje błąd uniemożliwiający wyświetlenie Hoppera dla użytkownika (tak, żeby sobie mógł popatrzeć). Do naprawy tego błędu trzeba znaleźć plik mujoco_rendering.py i zmienić nazwę zmiennej:
- ścieżka to (od folderu z Anacondą) Anaconda3\envs\"nazwa środowiska w anacondzie"\Lib\site-packages\gymnasium\envs\mujoco
- w linijce nr 593 zmienić "self.data.solver_iter" na "self.data.solver_niter" (dopisać "n")
- Debugger jupytera podpowiada powyższą ścieżkę wraz z lokalizacją w pliku.

## Tworzenie środowiska

In [1]:
# Przygotowanie bibliotek
import os as os # biblioteka do tworzenia ścieżki zapisu i odczytu
import gymnasium as gym # gymansium, baza środowisk
from stable_baselines3 import PPO # agent PPO
from stable_baselines3 import DDPG # agent DDPG
from stable_baselines3.common.evaluation import evaluate_policy # funkcja do ewaluacji agentów

Załadowanie środowiska po raz pierwszy sprawia, że można wyświetlać to, co się tam dzieje i można w nim uczyć agentów. Jednak po zamknięciu środowiska nie będzie nic wyświetlane, dopóki nie zostanie załadowane od nowa. Wszystkie inne operacji m. in. uczenie nadal można przeprowadzać. Trzeba o tym pamiętać jeśli chce się oglądać agenta, ale też warto jest zamknąć środowisko przed rozpoczęciem uczenia, żeby nie wyświetlało każdego podejścia agenta.

In [2]:
# Załadowanie środowiska z gymnasium
env = gym.make('Hopper-v4', render_mode="human")

In [7]:
# Zamknięcie środowiska
env.close()

In [ ]:
# Test. Jeśli wszystko zostało poprawnie ustawione, uruchomi się środowisko z Hopperem wykonującym losowe akcje.
observation, info = env.reset(seed=0)

for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    frame = env.render()

    if terminated or truncated:
        observation, info = env.reset()

env.close()

## Tworzenie i uczenie agentów

Parametr verbose powoduje pisanie loga podczas uczenia. W zależności od parametru total_timesteps, uczenie zajmie mniej lub więcej czasu, a wyświetlany log pokazuje ile takich jednostek czasu upłynęło, co można odebrać jako postęp uczenia (dla potrzeby szacowania *"ile to jeszcze może zająć"*).

In [ ]:
# Stworzenie agenta i jego uczenie (agent PPO)
agentPPO = PPO("MlpPolicy", env, verbose=1)
agentPPO.learn(total_timesteps=200000) # UWAGA! Uczenie zajmuje dużo czasu! (u mnie 200000 zajęło od 10 do 15 minut)

In [ ]:
# Stworzenie agenta i jego uczenie (agent DDPG)
agentDDPG = DDPG("MlpPolicy", env, verbose=1)
agentDDPG.learn(total_timesteps=200000) # UWAGA! Uczenie zajmuje BARDZO dużo czasu! (u mnie 200000 zajęło ok. 5 godzin)

## Zapisanie i ładowanie parametrów agentów

In [3]:
# Utworzenie ścieżki zapisu (wg poniższego zapisu modele muszą znajdować się w folderze o nazwie "agents", który znajduje się obok tego Jupytera)
pathPPO = os.path.join('agents', 'PPO')
pathDDPG = os.path.join('agents', 'DDPG')

In [ ]:
# Zapis parametrów agentów
agentPPO.save(pathPPO)
agentDDPG.save(pathDDPG)

In [4]:
# Załadowanie parametrów agentów
agentPPO = PPO.load(pathPPO, env=env)
agentDDPG = DDPG.load(pathDDPG, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


## Testowanie i obserwacja

In [5]:
# Testowanie/Ewaluacja agenta. Funkcja ewaluacji zwraca kolejno średnią nagrodę i odchylenie standardowe
# agent PPO
evaluate_policy(agentPPO, env, n_eval_episodes=3, render=True)

C:\Users\spryt\Anaconda3\envs\pytorch\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(2270.486060063044, 9.565386642580773)

In [6]:
# agent DDPG
evaluate_policy(agentDDPG, env, n_eval_episodes=10, render=True)

C:\Users\spryt\Anaconda3\envs\pytorch\Lib\site-packages\glfw\__init__.py:916: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


(1098.071695381403, 613.2127775657158)

## Kod alternatywny do obserwacji

In [7]:
# Obserwacja agenta PPO
observation, info = env.reset(seed=0)

for _ in range(1000):
    action, observation = agentPPO.predict(observation, info) # wykorzystanie PPO do wykonania akcji
    observation, reward, terminated, truncated, info = env.step(action)
    frame = env.render()

    if terminated or truncated:
        observation, info = env.reset()

#env.close()

In [ ]:
# Obserwacja agenta DDPG
observation, info = env.reset(seed=0)

for _ in range(1000):
    action, observation = agentDDPG.predict(observation, info) # wykorzystanie DDPG do wykonania akcji
    observation, reward, terminated, truncated, info = env.step(action)
    frame = env.render()

    if terminated or truncated:
        observation, info = env.reset()

#env.close()